In [44]:
!nvidia-smi

Sat Dec  9 10:17:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  | 00000000:24:00.0  On |                  N/A |
|  0%   24C    P8               9W / 200W |    507MiB / 12282MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [45]:
import pickle
from sklearn.model_selection import train_test_split

In [46]:
lyrics = pickle.load(open('french_lyrics_only_v2.pickle', 'rb'))

In [47]:
song_lyrics = ['\n'.join(lines) for artist, songs in lyrics.items() for title, lines in songs.items()]

In [48]:
print(song_lyrics[1])

Et nous, plébéiens d'à-présent
Anciens Egyptiens, nouveaux roturiers bohémiens
En aspirant à la justice, nous n’aspirâmes qu'au statut d'humain

Et nous demain, vilains d’aujourd'hui, manants d'avant-hier
Fils honni par ses propres pères
En aspirant à l'égalité, nous n'aspirâmes qu'au statut d'humain

Et nous, prolétaires en bas de l'échelle alimentaire
Animal servile se rêvant prince de la ville
En aspirant à la fraternité, nous n’aspirâmes qu’au statut d'humain
C’est comme si ma peau finissait jaunie à cause de mon gilet noir
Le malheur ne discrimine point, te voilà maintenant parlant de moi
Ô ciel, quelle méprisable condition qu'est devenue la mienne
Je ne peux plus dire à qui que ce soit que j'aime
C'est comme si ma peau finissait jaunie à cause de mon gilet noir
Le malheur ne discrimine point, te voilà maintenant parlant de moi
C’est la vie, c'est la France, c'est le monde
C'est la fronde, la galère qui nous poussent à l'extrême
C'est l'enfance qu'on regrette, plus personne dit je

In [43]:
train, test = train_test_split(song_lyrics,test_size=0.15)

In [ ]:
def build_text_files(data_text, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for text in data_text:
        data += text + ' '
    f.write(data)